In [7]:
import math
import numpy as np
import gurobipy as GRB
import gurobipy as gur
from gurobipy import GRB,quicksum

n=8
size=100
K=166

w=[7,11,15,19,23,27,35,42]
b=[52,97,140,191,85,129,26,20]
pi=[0 for i in range(K)]



I=len(w)






#producing an initial feasible solution and PI
st=[0,0,0,0,0,0,0,0]
df=[]
for i in range(len(w)):
    st=[0,0,0,0,0,0,0,0]
    e=int(100/w[i])
    st[i]=e
    st2=[st for i in range(math.ceil(b[i]/e))]
    for stock in st2:
        df.append(stock)
len(df)


K = 166 #obtained from the initial feasible solution in Subgradient code
f = [1 for i in range(0, K)]
pi=[0 for i in range(K)]


F=[]
F.append(f)
F
H=[]
H.append(df)


UB=1000
LB=-1000
iterations=0
while UB-LB>0.0001:
    iterations+=1
    #SUBPROBLEM 1
    subproblem1=gur.Model()
    y=subproblem1.addVars(K,vtype=GRB.BINARY)
    subproblem1.setObjective(quicksum(y[k] for k in range(K))-100*quicksum(y[k]*pi[k] for k in range(K)),GRB.MINIMIZE)
    subproblem1.optimize()
    subproblem1.objVal


    #SUBPROBLEM 2
    subproblem2=gur.Model()
    x=subproblem2.addVars(len(w),K,vtype=GRB.INTEGER)
    objective=0
    for i in range(len(w)):
        objective+=w[i]*quicksum(pi[k]*x[i,k] for k in range(K))    
    subproblem2.setObjective(objective,GRB.MINIMIZE)
    for i in range(len(w)):
        subproblem2.addConstr(quicksum(x[i,k] for k in range(K) )==b[i])
    subproblem2.optimize()
    subproblem2.objVal


    #OBTAINING THE LB, ATTACHING THE GENERATED X AND Y TO THE H
    L_pi=subproblem1.objVal+subproblem2.objVal
    if L_pi>LB:
        LB=L_pi


    f = []
    for k in range(K):
        f.append(y[k].x)
    F.append(f)
    df = np.array([[0 for j in range(0, I)] for u in range(0, K)])
    for k in range(K):
        for i in range(I):
            df[k][i] = x[i, k].x
    H.append(df)



    #DUAL MASTER PROBLEM
    master=gur.Model()
    theta1=master.addVar(vtype=GRB.CONTINUOUS) #determines whether stock k is used or not
    theta2=master.addVars(len(w),vtype=GRB.CONTINUOUS) #determines whether stock k is used or not


    pi=master.addVars(K,vtype=GRB.CONTINUOUS)


    master.setObjective(theta1+quicksum(theta2[i] for i in range(len(w))) ,GRB.MAXIMIZE)
    for f in F:
        master.addConstr(theta1<=quicksum(f[k] for k in range(K))-100*quicksum(f[k]*pi[k] for k in range(K)))


    for h in H:
        for i in range(len(w)):
            master.addConstr(theta2[i]<=w[i]*quicksum(h[k][i]*pi[k] for k in range(K) ))



    master.optimize()


    UB=master.objVal


    #UPDATING THE PI
    pij=[]
    for i in range(len(pi)):
        pij.append(pi[i].x)
    #pi=[pi.x for i in range(pi)]
    pi=pij


















Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 166 columns and 0 nonzeros
Model fingerprint: 0xcb19c01c
Variable types: 0 continuous, 166 integer (166 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0x0260ce7f
Variable types: 0 continuous, 1328 integer (0

Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 45 rows, 175 columns and 425 nonzeros
Model fingerprint: 0xc749dfd0
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+02]
Presolve removed 2 rows and 148 columns
Presolve time: 0.00s
Presolved: 43 rows, 27 columns, 126 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8725496e+02   4.420854e+01   0.000000e+00      0s
      19    1.5609000e+02   0.0

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 166 columns and 0 nonzeros
Model fingerprint: 0xcad75041
Variable types: 0 continuous, 166 integer (166 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [7e-01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -154.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: -154 
No other solutions better than -154

Optimal solution found (tolerance 1.00e-04)
Best objective -1.540000000000e+02, best bound -1.540000000000e+02, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0xee5f1e14
V

  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-02, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 204.9714286
Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 108 rows, 175 columns and 628 nonzeros
Model fingerprint: 0xa5ade2db
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+02]
Presolve removed 2 rows and 115 columns
Presolve time: 0.00s
Presolved: 106 rows,

       0    2.8725496e+02   1.262029e+02   0.000000e+00      0s
      35    1.5158146e+02   0.000000e+00   0.000000e+00      0s

Solved in 35 iterations and 0.01 seconds
Optimal objective  1.515814615e+02
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 166 columns and 0 nonzeros
Model fingerprint: 0x232116f2
Variable types: 0 continuous, 166 integer (166 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [8e-02, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -145.3076923

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: -145.308 
No other solutions better than -145.308

Optimal solution found (tolerance 1.00e-04)
Best objective -1.453076923077e+02, best bound -1.453076923077e+02, gap 0.0000%
Gurobi Optim

...

Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 114.971 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.149711898108e+02, best bound 1.149711898108e+02, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 945 rows, 175 columns and 10106 nonzeros
Model fingerprint: 0x77bf807a
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 944 rows, 174 columns, 10001 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8725496e+02   8.547222e+02   0.000000e+00      0s
     554    1.4366478e+02   0.000000e+00   0.000000e+00      

Optimal objective  1.436384993e+02
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 166 columns and 0 nonzeros
Model fingerprint: 0xbb829982
Variable types: 0 continuous, 166 integer (166 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e-03, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -1.9562517

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: -1.95625 
No other solutions better than -1.95625

Optimal solution found (tolerance 1.00e-04)
Best objective -1.956251663250e+00, best bound -1.956251663250e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328

Model fingerprint: 0xdcad878a
Variable types: 0 continuous, 1328 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-02, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 143.4800000
Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 114.276 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.142764794485e+02, best bound 1.142764794485e+02, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1008 rows, 175 columns and 11042 nonzeros
Model fingerprint: 0x688a087e
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00

Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 1034 rows, 174 columns, 11049 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8725496e+02   9.334544e+02   0.000000e+00      0s
     577    1.4348000e+02   0.000000e+00   0.000000e+00      0s

Solved in 577 iterations and 0.04 seconds
Optimal objective  1.434800000e+02
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 166 columns and 0 nonzeros
Model fingerprint: 0x0198f73b
Variable types: 0 continuous, 166 integer (166 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e-16, 4e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -1.9074621
Found heuristic solution: objective -1.9074621

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 availabl


Optimal solution found (tolerance 1.00e-04)
Best objective -9.913445595865e-01, best bound -9.913445595865e-01, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0x2680bc5f
Variable types: 0 continuous, 1328 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 143.4800000
Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 106.005 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.060049579505e+02, best bound 1.060049579505e+02, gap 0.0000%
Gurobi Optimizer version 9.1.1 b

In [16]:
# master.printAttr('X')
#theta2


{0: <gurobi.Var C1 (value 3.64)>,
 1: <gurobi.Var C2 (value 10.67)>,
 2: <gurobi.Var C3 (value 21.0)>,
 3: <gurobi.Var C4 (value 36.29)>,
 4: <gurobi.Var C5 (value 19.55)>,
 5: <gurobi.Var C6 (value 34.83)>,
 6: <gurobi.Var C7 (value 9.1)>,
 7: <gurobi.Var C8 (value 8.4)>}